### Make lists of image and label paths

In [1]:
import os
#dir_imgs = "/mnt/sda/deep_learning/CSE527_FinalProject-master/images/train"
dir_imgs = "/mnt/sda/deep_learning/phase_segmentation/images/train"
dir_labels = "/mnt/sda/deep_learning/phase_segmentation/images/label"
assert(os.path.isdir(dir_imgs))
assert(os.path.isdir(dir_labels))

In [11]:
import glob
img_paths = sorted(glob.glob(dir_imgs + "/*.tif"))
label_paths = sorted(glob.glob(dir_labels + "/*.tif"))
assert(len(img_paths) ==len(label_paths))
print(img_paths[1:10])
print(label_paths[1:10])

train_size = (556,556)
resize = False

['/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_003.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_005.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_007.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_009.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_011.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_013.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_015.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_019.tif', '/mnt/sda/deep_learning/phase_segmentation/images/train/10_M890_Nital_LOM_1000x_021.tif']
['/mnt/sda/deep_learning/phase_segmentation/images/label/10_M890_Nital_LOM_1000x_003.tif', '/mnt/sda/deep_learning/phase_segmentation/images/label/10_M890_Nital_LOM_1000x_005.tif', '/mnt/sd

### Utility functions

In [12]:
## load random image and label from list of image and label paths
import random
from PIL import Image
def load_rand_sample(image_paths, label_paths, shape=train_size, pil=False):
    num_samples = len(image_paths)
    for _ in range(num_samples):
        ii_rand = random.randint(0,num_samples-1)
        img_rand = Image.open(image_paths[ii_rand])
        label_rand = Image.open(label_paths[ii_rand])
        if resize:
            img_rand = img_rand.resize(shape)    
            label_rand = label_rand.resize(shape)
        if np.array(label_rand).sum()>150:
            break
    if not pil:
        return pil_to_np(img_rand, label_rand)
    else:
        return img_rand, label_rand

def pil_to_np(img_pil, mask_pil):
    img  = np.array(img_pil).astype("float32")/255.
    mask = np.array(mask_pil)
    return img, mask

""" Elastic transform code from https://gist.github.com/erniejunior/601cdf56d2b424757de5"""
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
def elastic_transform_git(image, alpha, sigma, order=0, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=order, mode='reflect')
    return distored_image.reshape(image.shape)

""" Elastic transform code from https://gist.github.com/erniejunior/601cdf56d2b424757de5"""
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
def elastic_transform_git_mod(image, label, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))
    
    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    distored_label = map_coordinates(label, indices, order=0, mode='reflect')
    return distored_image.reshape(shape), distored_label.reshape(shape)


import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import albumentations as A

### Test brightness parameter

In [13]:
%matplotlib inline
img, _ = load_rand_sample(img_paths, label_paths)
@interact_manual
def adjust_brightness(brightness_factor=(-1, 1, 0.05)):
    aug = A.RandomBrightness(p=1)
    img_adj = aug.apply(img, beta=brightness_factor)
    #plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    plt.subplot(122)
    plt.imshow(img_adj, cmap="gray", vmin=0, vmax=1)
    print("sum img:", img.sum())
    print("sum img_adj:", img_adj.sum())

interactive(children=(FloatSlider(value=0.0, description='brightness_factor', max=1.0, min=-1.0, step=0.05), B…

In [8]:
brightness_factor = (-0.1, 0.1)

### Test contrast parameter

In [14]:
img, _ = load_rand_sample(img_paths, label_paths)
@interact_manual
def adjust_contrast(contrast_factor=(0,2,0.05)):
    aug = A.RandomContrast(p=1)
    img_adj = aug.apply(img, alpha=contrast_factor)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    plt.subplot(122)
    plt.imshow(img_adj, cmap="gray", vmin=0, vmax=1)

interactive(children=(FloatSlider(value=1.0, description='contrast_factor', max=2.0, step=0.05), Button(descri…

In [7]:
contrast_factor = (0.85, 1.15)

### Test affine transformation parameters

In [15]:
img, label = load_rand_sample(img_paths, label_paths)
@interact_manual
def affine_transform(angle=(-30,30,1), translate_x=(-0.2,0.2,0.05), translate_y=(-0.2,0.2,0.05), scale=(0.9,1.1,0.05)):
    aug = A.ShiftScaleRotate(border_mode=0)
    img_trf = aug.apply(img, angle=angle, scale=scale, dx=translate_x, dy=translate_y)
    label_trf = aug.apply_to_mask(label, angle=angle, scale=scale, dx=translate_x, dy=translate_y)
    plt.figure(figsize=(25,25))
    plt.subplot(121)
    plt.imshow(img, cmap="gray", vmin=0, vmax=1)
    plt.imshow(label, alpha=0.3)
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray", vmin=0, vmax=1)
    plt.imshow(label_trf, alpha=0.3)

interactive(children=(IntSlider(value=0, description='angle', max=30, min=-30), FloatSlider(value=0.0, descrip…

In [9]:
angle=30
#translate = (50, 50)
translate_x = 0.1
translate_y = 0.1
scale = (0.9,1.1)
#shear = 6

In [10]:
%%timeit
img_trf = TF.affine(img, angle, translate, scale, shear, resample=0, fillcolor=None)
label_trf = TF.affine(label, angle, translate, scale, shear, resample=0, fillcolor=None)

NameError: name 'TF' is not defined

In [11]:
%%timeit
aug = A.Compose([A.ShiftScaleRotate(border_mode=0)])
augmented = aug(image=img, mask=label)

330 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
%%timeit
aug = A.ShiftScaleRotate(border_mode=0)
img_trf = aug.apply(img, angle=angle, scale=scale, dx=translate_x, dy=translate_y)
label_trf = aug.apply_to_mask(label, angle=angle, scale=scale, dx=translate_x, dy=translate_y)

2.94 ms ± 58.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Test elastic transform parameters

In [25]:
img, label = load_rand_sample(img_paths, label_paths)
@interact_manual
def elastic_transform(alpha=(0,100,1), sigma=(5,7,0.01)):
    aug = A.Compose(
        [A.ElasticTransform(p=1, border_mode=4, alpha=alpha, alpha_affine=0, sigma=sigma, approximate=False, interpolation=0)])
    augmented = aug(image=img, mask=label)
    label_trf = augmented["mask"]
    img_trf = augmented["image"]
    plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    print(img_trf.max(), img_trf.min(), img.max(), img.min())
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(IntSlider(value=50, description='alpha'), FloatSlider(value=6.0, description='sigma', ma…

In [82]:
alpha_affine = 0
alpha = 40
sigma = 6

In [83]:
%%timeit
aug = A.Compose(
        [A.ElasticTransform(p=1, border_mode=0, alpha=alpha, alpha_affine=alpha_affine, sigma=sigma)])
augmented = aug(image=img, mask=label)

140 ms ± 753 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
%%timeit
aug = A.Compose(
        [A.ElasticTransform(p=1, border_mode=0, alpha=alpha, alpha_affine=alpha_affine, sigma=sigma, approximate=True)])
augmented = aug(image=img, mask=label)

57.8 ms ± 599 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [118]:
%%timeit
img_2, label_2 = elastic_transform_git_mod(np.array(img)[..., None], np.array(label)[..., None], alpha, sigma)

76.2 ms ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Test optical distortion parameters

In [31]:
img, label = load_rand_sample(img_paths, label_paths)
%matplotlib inline
@interact_manual
def optical_distortion(distort_limit=(0,1,0.05), shift_limit=(0,1,0.05)):
    aug = A.OpticalDistortion(p=1, border_mode=4, interpolation=0)
    img_trf = aug.apply(img, k=distort_limit, dx=shift_limit, dy=shift_limit)
    label_trf = aug.apply_to_mask(label, k=distort_limit, dx=shift_limit, dy=shift_limit)
    plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(FloatSlider(value=0.0, description='distort_limit', max=1.0, step=0.05), FloatSlider(val…

In [21]:
distort_limit = 0.1
shift_limit = 0.5

### Test blurring

In [23]:
img, label = load_rand_sample(img_paths, label_paths)
%matplotlib notebook
@interact_manual
def blur(blur_limit=(3,20,1)):
    aug = A.Blur(p=1)
    img_trf = aug.apply(img, ksize=blur_limit)
    label_trf = label
    #plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(IntSlider(value=11, description='blur_limit', max=20, min=3), Button(description='Run In…

In [24]:
blur_limit = 3

### Test gaussian blurring

In [27]:
%matplotlib notebook
img, label = load_rand_sample(img_paths, label_paths)
@interact_manual
def gauss_blur(blur_limit=(3,20,1)):
    aug = A.GaussianBlur(p=1)
    img_trf = aug.apply(img, ksize=blur_limit)
    label_trf = label
    #plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(IntSlider(value=11, description='blur_limit', max=20, min=3), Button(description='Run In…

In [28]:
blur_limit = 7

### Test gaussian noise

In [40]:
%matplotlib notebook
img, label = load_rand_sample(img_paths, label_paths)
@interact_manual
def gauss_noise(var_limit=(0,0.02,0.005)):
    aug = A.Compose([A.GaussNoise(p=1, var_limit=var_limit)])
    img_trf = aug(image=img)["image"]
    label_trf = label
    #plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(FloatSlider(value=0.01, description='var_limit', max=0.02, step=0.005), Button(descripti…

In [41]:
var_limit = 0.015

## Test motion blur

In [43]:
%matplotlib notebook
img, label = load_rand_sample(img_paths, label_paths)
@interact_manual
def motion_blur(blur_limit=(3,10,1)):
    aug = A.Compose([A.MotionBlur(p=1, blur_limit=blur_limit)])
    img_trf = aug(image=img)["image"]
    label_trf = label
    #plt.figure(figsize=(30,15))
    plt.subplot(122)
    plt.imshow(img_trf, cmap="gray")
    plt.imshow(label_trf, alpha=0.3)
    plt.subplot(121)
    plt.imshow(img, cmap="gray")
    plt.imshow(label, alpha=0.3)

interactive(children=(IntSlider(value=6, description='blur_limit', max=10, min=3), Button(description='Run Int…

In [44]:
blur_limit = 3